In [ ]:
# Создание и преобразование столбцов с помощью функций
# .apply() - метод
# функция, которая принимает на вход один элемент столбца, каким-то образом его обрабатывает и возвращает результат, после чего применить эту функцию к каждому элементу 

In [1]:
import pandas as pd
melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')
# display(melb_data.head())
melb_df = melb_data.copy()
# display(melb_df.head())
melb_df = melb_df.drop(['index', 'Coordinates'], axis=1)
#melb_df.head()
total_rooms = melb_df['Rooms'] + melb_df['Bathroom'] + melb_df['Bedroom']
#display(total_rooms)
melb_df['MeanRoomsSquare'] = melb_df['BuildingArea']/total_rooms
#display(melb_df['MeanRoomsSquare'])
diff_area = melb_df['BuildingArea'] - melb_df['Landsize']
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
melb_df['AreaRatio'] = diff_area/sum_area
#display(melb_df['AreaRatio'])
price_square = melb_df['Price']**2
#display(price_square)

melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
# print(melb_df['Date'])

year_sold = melb_df['Date'].dt.year
#print('год продажи', year_sold)
#print('min yearsold', year_sold.min())
#print('max yearsold', year_sold.max())
#print('mode of sold', year_sold.mode()[0])

melb_df['MonthSale'] = melb_df['Date'].dt.month
melb_df['MonthSale'].value_counts(normalize=True)

melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']
#display(melb_df['AgeBuilding'])
melb_df = melb_df.drop(['YearBuilt'], axis=1)
#display(melb_df.head())

melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek
weekend_count = melb_df[(melb_df['WeekdaySale'] == 5) | (melb_df['WeekdaySale'] == 6)].shape[0]
#print(weekend_count, 'объектов недвижимости было продано в выходные')


In [3]:
# В наших данных есть столбец с адресами объектов недвижимости
# check количество уникальных значений в столбце с помощью метода nunique()
print('количество уникальных значений в столбце', melb_df['Address'].nunique())

количество уникальных значений в столбце 13378


In [4]:
# давайте извлечём из признака адреса характеристику подтипа улицы (улица, шоссе, авеню, бульвар)
# Для этого сначала внимательнее посмотрим на структуру адреса, выберем несколько строк столбца Address
print(melb_df['Address'].loc[177])
print(melb_df['Address'].loc[1812])
print(melb_df['Address'].loc[9001])

2/119 Railway St N
9/400 Dandenong Rd
172 Danks St


In [ ]:
# Для того чтобы выделить подтип улицы, на которой находится объект, можно использовать следующую функцию:
# На вход данной функции поступает строка с адресом.
def get_street_type(address):
    exclude_list = ['N', 'S', 'W', 'E'] # Создаём список географических пометок exclude_list.
    address_list = address.split('') # Метод split() разбивает строку на слова по пробелу, получаем список слов в строке 
    street_type = address_list[-1] # Обрезаем список, оставляя в нём только последний элемент, потенциальный подтип улицы
    